In [1]:
import pickle, tqdm, time, os, tensorflow_hub as hub, numpy as np, pandas as pd

In [2]:
# load & cache tensorflow model
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")
embed(['testing'])
print('model cached')

model cached


In [3]:
# load data df & embeddings
df = pd.read_pickle('data/clean_data.pkl')
title_embeddings = np.load('data/title_embeddings.npz')['a']
# text_embeddings = np.load('data/texts_embeddings.npz')['a']


**STEPS:**
1. Query --> query_vector = Q
2. Query --> find similar docs, return top N most similar docs
3. User Faves Doc A --> Replace Q with Q' = Q + (K)(A - Q) where K is param between 0 & 1 we choose
3. User Hides Doc B --> Replace Q' with Q'' = (1 + K)(Q') - (K)(B)

In [41]:
SENSITIVITY = 0.6
EMBED_TYPE = 'titles'

In [42]:
def getMostSimilar(query_vector, embeds=title_embeddings, docsdf=df.copy(),filter_bad=True):
    scores = query_vector.dot(embeds.T).flatten()
    docsdf['similarity_score'] = scores
    docsdf = docsdf.sort_values('similarity_score', ascending=False).copy()
    return docsdf

In [43]:
def update_embedding(q_vector, feedback_vector, feedback):
    # New q vector = (1 - feedback*SENSITIVITY)*(q_vector)  + feedback*SENSITIVITY*(feedback_vector)
    new_q = (1 - feedback*SENSITIVITY)*q_vector + feedback*SENSITIVITY*feedback_vector
    return new_q

In [44]:
def giveFeedback(post_id, feedback):
    feedback_doc = df[df['ids'] == post_id][EMBED_TYPE].values[0]
    feedback_vector = embed([feedback_doc]).numpy()
    new_q = update_embedding(Q, feedback_vector, feedback)
    return new_q

In [45]:
QUERY = 'passwords for wifi'
Q = embed([QUERY]).numpy()

In [47]:
sim = getMostSimilar(Q)
sim.head(50)

,ids,titles,texts,similarity_score
662052,8dv3bq,WIBTA for changing the WiFi password?,My husband and I live with another couple and ...,0.641542
403539,ddymcy,WIBTA for changing the wifi password?,My best friend and I share a small apartment. ...,0.641542
278717,f0k537,WIBTA for not wanting to give my neighbors the...,I've let my neighbors use my wifi for over a y...,0.581597
217069,fxq2sb,WIBTA for changing my wifi's password so my ne...,"where im from,the wifi is generally trash. to ...",0.572575
486630,cfswme,WIBTA if I didn't share my wifi password with ...,Throwaway account because reasons.\n\n&amp;#x2...,0.566454
458289,cpzws3,WIBTA Changing wifi password in revenge,"Hi! This is my first post on here, and I'm fai...",0.564805
177649,gj77cv,AITA for changing my wifi password?,"Around August of last year, my upstairs neighb...",0.561346
290874,eubzr0,AITA for not giving my neighbors our WiFi pass...,I was out on our apartment breezeway looking a...,0.558386
341920,e82ljr,AITA for not giving out my wifi password?,My ex's friend lives by me and lost her intern...,0.553269
625301,ap1h08,AITA For changing Wifi Password on Roommate?,Tonight my roommate refused to pay wifi becaus...,0.547110


In [48]:
Q = giveFeedback(post_id='ap1h08', feedback=1)

In [49]:
sim = getMostSimilar(Q)
sim.head(5)

,ids,titles,texts,similarity_score
625301,ap1h08,AITA For changing Wifi Password on Roommate?,Tonight my roommate refused to pay wifi becaus...,0.818844
63571,i5dsnd,AITA for changing the wifi password on housemate?,"Hey Reddit, a bit of background on this:\n\nI ...",0.795251
529726,by85cy,AITA For changing the wifi password without te...,I live in an apartment with one other roommate...,0.781137
400772,dfbyhf,AITA for not giving my roommate my Wifi password?,"To give a little background info, I’m a grad s...",0.773663
578627,bd2r9g,AITA for changing the WiFi SSID and password o...,Obligatory first post ever.\n\nI'm a 20 year o...,0.761273
